In [1]:
from automata import *

In [ ]:
Q = ['q0', 'q1', 'q2']#'q1', 'q2'
A = ['a1', 'a2', ] #'a3'
B = ['0', '1']
phi = {
    ('q0', A[0]): 'q0',
    ('q0', A[1]): 'q1',#'q1',
    #('q0', A[2]): 'q1',

    ('q1', A[0]): 'q1',
    ('q1', A[1]): 'q2',
    #('q1', A[2]): 'q2',
#
    ('q2', A[0]): 'q2',  
    ('q2', A[1]): 'q2', 
    #('q2', A[2]): 'q2', 


}

psi = {
    ('q0', A[0]): B[1],
    ('q0', A[1]): B[0],
    #('q0', A[2]): B[1],
#
    ('q1', A[0]): B[0],
    ('q1', A[1]): B[0],
    #('q1', A[2]): B[1],
#
    ('q2', A[0]): B[0],
    ('q2', A[1]): B[1],
    #('q2', A[2]): B[0],

}
q_start = 'q0'

In [ ]:
schema_encoding = {
    # 'a1': '1',
    # 'a2': '01',
    # 'a3': '100',
    # 'a4': '0100',
    # 'a5': '0000',

    #A[0]: '01',
    #A[1]: '0101',
    #A[2]: '010101',

    'a1': '0',
    'a2': '1',
    #'a3': '000',
}
